In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from datetime import datetime
import time
import random
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [2]:
# Load the DataFrames
df = pd.read_csv('exports/stocks.csv')
df

,stock_name,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250
1,ABF MALAYSIA BOND INDEX FUND,0800EA
2,ABLE GLOBAL BERHAD,7167
3,ABLEGROUP BERHAD,7086
4,ABM FUJIYA BERHAD,5198
...,...,...
1008,YKGI HOLDINGS BERHAD,7020
1009,YLI HOLDINGS BERHAD,7014
1010,YNH PROPERTY BERHAD,3158
1011,YONG TAI BERHAD,7066


In [3]:
# Put it into a list called stock_code
stock_code = list(df['stock_code'])
stock_code

['5250',
 '0800EA',
 '7167',
 '7086',
 '5198',
 '03028',
 '7131',
 '0218',
 '0122',
 '1481',
 '5281',
 '9148',
 '7191',
 '7146',
 '0181',
 '6599',
 '5139',
 '5185',
 '7145',
 '0258',
 '7315',
 '7078',
 '0209',
 '5238',
 '2658',
 '7609',
 '5116',
 '5115',
 '2674',
 '0079',
 '2488',
 '1163',
 '03051',
 '5269',
 '5127',
 '5293',
 '5307',
 '5120',
 '03011',
 '1015',
 '7031',
 '6351',
 '7083',
 '0048',
 '4758',
 '0226',
 '6556',
 '5082',
 '5568',
 '5088',
 '7090',
 '5015',
 '6432',
 '0119',
 '7214',
 '7181',
 '7007',
 '0038',
 '0068',
 '7722',
 '7129',
 '0159',
 '0105',
 '4057',
 '7162',
 '7054',
 '03032',
 '6399',
 '0072',
 '8176',
 '7048',
 '5130',
 '7099',
 '5302',
 '03037',
 '8885',
 '5204',
 '7579',
 '6888',
 '5106',
 '7120',
 '2305',
 '5021',
 '7005',
 '03012',
 '0098',
 '5258',
 '7251',
 '1899',
 '6602',
 '0187',
 '5190',
 '3239',
 '3395',
 '5196',
 '4219',
 '5248',
 '9814',
 '7668',
 '0263',
 '6173',
 '5932',
 '0195',
 '6998',
 '5032',
 '0179',
 '5069',
 '0168',
 '9288',
 '7036',
 '

In [5]:
# Create an empty dataframe
quarter_rep=pd.DataFrame()
no_scrape=[]

In [6]:
# Connect/open to Chrome webdriver 
driver = webdriver.Chrome()
driver.implicitly_wait(30)

for s in stock_code:
    try:
        url='https://www.klsescreener.com/v2/stocks/view/'+s
        driver.get(url)
        content = driver.page_source
        soup=bs(content)

        # Scrap table
        tables = soup.find_all('table')
        # Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        # Find the table that contain column 'ROE'
        i = 0
        has_found = False
        while i < len(dfs) and not has_found:
            n = 0
            while n < len(dfs[i].columns):
                if dfs[i].columns[n] == 'ROE':
                    has_found = True
                n += 1
            i += 1
            
        if has_found:
            df=dfs[i-1]
            
            # Extract all the columns needed
            df2=df[['EPS','DPS','NTA','Revenue','P/L','Quarter','Q Date','Financial Year','Announced','ROE']]
            
            # Changes data types 
            df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
            df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
            
            # Filter data between two dates  
            filtered_df = df2.loc[(df2['Financial Year'] >= '2018-01-01')
                          & (df2['Financial Year'] < '2023-12-31')]
            filtered_df.reset_index(drop=True, inplace=True)
            
            if filtered_df.empty:
                no_scrape.append(s)
                print("Stock code :"+s+" has no  quarter report between 2018 to 2022")
            else:
                # Drop the date row 
                filtered_df.drop(filtered_df[filtered_df['EPS'] == filtered_df['Revenue']].index, inplace = True)
                filtered_df.reset_index(drop=True, inplace=True)

                # Add stock code
                filtered_df['Stock Code']=s

                # add year column
                filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")

                # Append it into quarter_rep dataframe
                quarter_rep = quarter_rep.append(filtered_df, ignore_index=True)
                print("Stock code :"+s+" yes")
            
        else:
            no_scrape.append(s)
            print("Stock code :"+s+" has no  annual report")
    except:
        no_scrape.append(s)
        print("Stock code :"+s+" cannot scrap")
        
    # Program to generate a random number between 5 to 30
    # Importing the random module
    sec = random.randint(1, 5)

    # Program scrapping waiting time
    time.sleep(sec)  # Seconds


C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Stock code :5250 yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Stock code :0800EA yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

...

Stock code :3158 yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Stock code :7066 yes


C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
C:\Users\Aidan\AppData\Local\Temp\ipykernel_5596\3628963292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

Stock code :5159 yes


In [7]:
quarter_rep

,EPS,DPS,NTA,Revenue,P/L,Quarter,Q Date,Financial Year,Announced,ROE,Stock Code,Year
0,0.24,0.000,0.1216,992.43m,2.65m,4,2022-12-31,2022-12-31,2023-02-28,2.0%,5250,2022
1,1.21,0.000,0.1106,988.21m,13.66m,3,2022-09-30,2022-12-31,2022-11-29,10.9%,5250,2022
2,2.29,0.000,0.0984,943.67m,25.77m,2,2022-06-30,2022-12-31,2022-08-25,23.3%,5250,2022
3,2.16,0.000,0.1015,839.98m,24.38m,1,2022-03-31,2022-12-31,2022-05-26,21.3%,5250,2022
4,2.60,0.000,0.0799,795.06m,29.24m,4,2021-12-31,2021-12-31,2022-02-24,32.5%,5250,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
19889,3.74,0.000,1.3700,55.77m,5.98m,1,2018-09-30,2019-06-30,2018-11-27,2.7%,5159,2019
19890,3.47,2.000,1.3306,50.22m,5.55m,4,2018-06-30,2018-06-30,2018-08-28,2.6%,5159,2018
19891,5.28,2.000,1.3159,54.68m,8.45m,3,2018-03-31,2018-06-30,2018-05-24,4.0%,5159,2018
19892,4.15,3.000,1.2631,51.84m,6.64m,2,2017-12-31,2018-06-30,2018-02-27,3.3%,5159,2018


In [8]:
# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/quarterreportscheck.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
quarter_rep.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/quarterreportscheck.csv


In [3]:
df2 = pd.read_csv('exports/quarterreportscheck.csv')
df2

,EPS,DPS,NTA,Revenue,P/L,Quarter,Q Date,Financial Year,Announced,ROE,Stock Code,Year
0,0.24,0.0,0.1216,992.43m,2.65m,4,2022-12-31,2022-12-31,2023-02-28,2.0%,5250,2022
1,1.21,0.0,0.1106,988.21m,13.66m,3,2022-09-30,2022-12-31,2022-11-29,10.9%,5250,2022
2,2.29,0.0,0.0984,943.67m,25.77m,2,2022-06-30,2022-12-31,2022-08-25,23.3%,5250,2022
3,2.16,0.0,0.1015,839.98m,24.38m,1,2022-03-31,2022-12-31,2022-05-26,21.3%,5250,2022
4,2.60,0.0,0.0799,795.06m,29.24m,4,2021-12-31,2021-12-31,2022-02-24,32.5%,5250,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
19889,3.74,0.0,1.3700,55.77m,5.98m,1,2018-09-30,2019-06-30,2018-11-27,2.7%,5159,2019
19890,3.47,2.0,1.3306,50.22m,5.55m,4,2018-06-30,2018-06-30,2018-08-28,2.6%,5159,2018
19891,5.28,2.0,1.3159,54.68m,8.45m,3,2018-03-31,2018-06-30,2018-05-24,4.0%,5159,2018
19892,4.15,3.0,1.2631,51.84m,6.64m,2,2017-12-31,2018-06-30,2018-02-27,3.3%,5159,2018
